## imports

In [2]:
import os
import numpy as np
from datetime import datetime
from tqdm import tqdm
from copy import deepcopy
from utils import multi_exp_data_to_fig

## constants

In [3]:
# hypothesis = 'pesto_protein_scores_ablation'
hypothesis = 'optimizer_batchsize_lr_gridsearch'
evaluation_type = 'validation'  # can be train / validation / test

## fetch data

In [4]:
experiments = os.listdir(f'../configurations/data/{hypothesis}')
experiments = [e.split('.json')[0] for e in experiments]

In [5]:
# Iterate over all experiments, per experiment - sort runs by time
# Going from most recent to oldest, take the first run of the expeirment that finished running (has inference results over all 5 folds)

run_paths = []
for experiment in experiments:
    experiment_runs = os.listdir(f'../results/patch_to_score/hypotheses/{hypothesis}/{experiment}')
    try:
        experiment_runs = sorted(experiment_runs, key=lambda x: datetime.strptime(x.split('_')[0], '%Y-%m-%d'), reverse=True)
    except Exception as e:
        print(e)
    experiment_valid_run_found = False
    for updated_experiment_run in experiment_runs:
        run_path = f'{hypothesis}/{experiment}/{updated_experiment_run}'
        if 'fold_4' in os.listdir(f'../results/patch_to_score/hypotheses/{run_path}'):
            run_paths.append(run_path)
            experiment_valid_run_found = True
            break
    if not experiment_valid_run_found:
        print(f'experiment: {experiment} didnt finish, skipping')
len(run_paths)

36

In [6]:
base_paths = [f'../results/patch_to_score/hypotheses/{path}' for path in run_paths]

## parse data

In [7]:
multi_exp_data = dict()
for base_path in tqdm(base_paths):
    exp_data = []
    exp_name = base_path.split('/')[-2]
    for fold in range(5):
        try:
            fold_labels = np.load(os.path.join(base_path, f'fold_{fold}', evaluation_type, 'labels.npy'))
            fold_predictions = np.load(os.path.join(base_path, f'fold_{fold}', evaluation_type, 'predictions.npy'))[:, 0]
            exp_data.append(np.stack([fold_labels, fold_predictions]))
        except Exception as e:
            pass
    multi_exp_data[exp_name] = exp_data

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:20<00:00,  1.76it/s]


## filter

In [8]:
filtered_multi_exp_data = deepcopy(multi_exp_data)
filtered_multi_exp_data = {exp_name: exp_data for exp_name, exp_data in filtered_multi_exp_data.items() if ('sgd' not in exp_name and '0.0001' not in exp_name)}

## plot

In [11]:
fig = multi_exp_data_to_fig(filtered_multi_exp_data, 
                            show_std_in_legend=True, log_scale=True, resolution=10_000)
# fig.show()

computing rocs


100%|██████████| 16/16 [00:00<00:00, 42.06it/s]

adding random roc
fig update
